In [ ]:
import os
from os import path
import sys
import pickle
from collections import namedtuple, defaultdict, Counter
from datetime import datetime, timedelta
from time import time
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from random import shuffle
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
import re
from sklearn.manifold import TSNE
import seaborn as sns
from sklearn.cluster import DBSCAN, KMeans

In [ ]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

pd.options.display.max_rows = 100

In [ ]:
dir_data = '/path/to/data/dir'
file_pcs = path.join(dir_data, 'patient_code_sequences.txt')
file_persons = path.join(dir_data, 'persons.csv')
file_concepts = path.join(dir_data, 'concepts.csv')
file_sequences = path.join(dir_data, 'patient_sequences.pkl')
file_backup_suffix = '.backup'

## Load data into dataframe

In [ ]:
# column indices for persons.csv
df_persons = pd.read_csv(file_persons, sep='\t', header=0, index_col=0, 
                         parse_dates=['birth_date'], infer_datetime_format=True)

# Check the data types of the columns
df_persons.dtypes

In [ ]:
# Load the concept definitions
df_concepts = pd.read_csv(file_concepts, sep='\t', header=0, index_col='concept_id')
df_concepts.head()

# Prep patient sequences into TaggedDocuments

In [ ]:
# Helpers for reading in the patient_code_sequences.txt

# Date of occurrence and list of concepts occurring on this date
DateOccurrence = namedtuple('DateOccurrence', ['date', 'concept_ids'])

def _process_pcs_line(line):
    """ Processes a line from patient_code_sequences.txt and parses out the patient ID
    and DateOccurrences """
    split = line.strip().split('\t')
        
    # person_id is the first entry
    pid = int(split.pop(0))
    
    # Process the remaining string into a list of Occurrences
    date_occurrences = [_process_date_occurrence_str(x) for x in split]
    
    return pid, date_occurrences

def _process_date_occurrence_str(dos):
    """ Processes a DateOccurrence string 
    format: YYYY-MM-DD:<list of concept IDs separated by commas> """
    date_str, concept_ids_str = dos.split(':')
    occ = DateOccurrence(datetime.strptime(date_str.strip(), '%Y-%m-%d'), 
                         [int(x) for x in concept_ids_str.split(',')])
    return occ

def create_patient_sequences(f_pcs_in, f_seq_out=None, min_seq_length=10, randomize_order=True, verbose=False, save_intermediates=False): 
    """ Reads the patient_code_sequences.txt file and parses it into sequences for each patient
    
    Note: save_intermediates makes it a lot slower """

    # For keeping track of processing time
    t1 = time()

    # pseqs - list of TaggedDocument(words=[concept_ids], tags=[person_id])
    pseqs = list()

    count = 0
    
    if f_seq_out:
        f_intermediate = f_seq_out + '.tmp'
    
    # Read patient_code_sequences.txt
    with open(f_pcs_in) as fh:  
        # Skip the heaer line
        fh.readline()
        
        for line in fh:
            # Parse the line into person_id and list of date_occurrences
            pid, date_occurrences = _process_pcs_line(line)

            # Combine sequence of concepts from each date into on sequence for the patient
            current_seq = []
            for date_occurrence in date_occurrences:
                concepts = date_occurrence.concept_ids
                if randomize_order:
                    # Randomize the order of concepts occurring on the same date. Shuffle is applied in place
                    shuffle(concepts)
                    
                current_seq += concepts
                
            if len(current_seq) >= min_seq_length:
                pseqs.append(TaggedDocument(words=[str(x) for x in current_seq], tags=[pid]))

            # Display progress
            count += 1
            if count % 100000 == 0:
                if verbose: 
                    # Processing time and size of data structure
                    ellapsed_time = (time() - t1) / 60
                    print(f'{count} - {ellapsed_time:.01f} min')

                if save_intermediates and f_seq_out:
                    # Save a backup copy of the data
                    pickle.dump(pseqs, open(f_intermediate, 'wb'), protocol=pickle.HIGHEST_PROTOCOL)      

    if f_seq_out:
        # Save the concept age distributions            
        pickle.dump(pseqs, open(f_seq_out, 'wb'), protocol=pickle.HIGHEST_PROTOCOL)

        # Delete the backup file
        if save_intermediates and path.exists(f_intermediate):
            os.remove(f_intermediate)

    # Display overall processing time
    ellapsed_time = (time() - t1) / 60
    print(f'{count} - {ellapsed_time:.01f} min')
    
    return pseqs

In [ ]:
pseqs = create_patient_sequences(file_pcs, f_seq_out=None, min_seq_length=5, randomize_order=True, 
                                        verbose=True, save_intermediates=False)
n_pseqs = len(pseqs)
print(n_pseqs)

### Train the models

In [ ]:
def model_filename(model, epochs=None):
    """ Generate a filename for to save the model using the string representation of the model, 
    which already includes most of the important model parameters. """
    f_model = re.sub('[^\w\-_\. ]', '_', str(model))
    if epochs:
        f_model += f'e{epochs}'
    f_model += '.d2v'
    return f_model

#### Paragraph Vector - Distributed Memory

In [ ]:
model_dm = Doc2Vec(dm=1, vector_size=100, window=7, min_count=5, alpha=0.023, hs = 0, negative=15, 
                   epochs=20, workers=6, report_delay=60)

# Build Vocab
t1 = time()
model_dm.build_vocab(pseqs, progress_per=1000000)
ellapsed_time = (time() - t1) / 60
print(f'Build Vocab Ellapsed Time: {ellapsed_time} min')

# Train
t1 = time()
model_dm.train(pseqs, total_examples=model_dm.corpus_count, epochs=model_dm.epochs, report_delay=60)
ellapsed_time = (time() - t1) / 60
print(f'Train Ellapsed Time: {ellapsed_time} min')

# Save the model
f_model = path.join(dir_data, model_filename(model_dm, epochs=model_dm.epochs))
print(f'Saving model to: {f_model}')
model_dm.save(f_model)

#### Paragraph Vector - Distributed Bag of Words

In [ ]:
model_dbow = Doc2Vec(dm=0, vector_size=100, window=7, min_count=5, alpha=0.023, hs = 0, negative=15, 
                     epochs=20, workers=6, report_delay=60)

# Build Vocab
t1 = time()
model_dbow.build_vocab(pseqs, progress_per=100000)
ellapsed_time = (time() - t1) / 60
print(f'Build Vocab Ellapsed Time: {ellapsed_time} min')

# Train
t1 = time()
model_dbow.train(pseqs, total_examples=model_dbow.corpus_count, epochs=model_dbow.epochs, report_delay=60)
ellapsed_time = (time() - t1) / 60
print(f'Build Vocab Ellapsed Time: {ellapsed_time} min')

# Save the model
f_model = path.join(dir_data, model_filename(model_dbow, epochs=model_dbow.epochs))
print(f'Saving model to: {f_model}')
model_dbow.save(f_model)

### Re-load the models
Reload the models if the kernel was shut down after training the models

In [ ]:
model_dm = Doc2Vec.load(path.join(dir_data, 'Doc2Vec_dm_m_d100_n15_w7_mc5_s0.001_t6_e20.d2v'))

In [ ]:
model_dbow = Doc2Vec.load(path.join(dir_data, 'Doc2Vec_dbow_d100_n15_mc5_s0.001_t6_e20.d2v'))

# Build the cohort for disease subtyping

In [ ]:
# Class for storing info about a cohort patient sequence
# Sequence stored as TaggedDocument object for D2V processing
# Will use OMOP concept ID for label
CohortPatientSeq = namedtuple('CohortPatientSeq', ['person_id', 'sequence', 'label', 'date_lower', 'date_upper'])

def create_cohort_patient_sequences(f_pcs_in, cohort_concepts, f_seq_out=None, time_window=180, randomize_order=True, verbose=False, save_intermediates=False): 
    """ Reads the patient_code_sequences.txt file and extracts sequences +/- <time_window> days around the 
    first occurrence of any encountered desired concept
    
    Note: save_intermediates makes it a lot slower """

    # For keeping track of processing time
    t1 = time()

    # pcss - cohort patient sequences: list of CohortPatientSeq objects
    cpss = list()
    count = 0
    
    # Time window for finding occurrences 
    time_window = timedelta(days=time_window)
    
    if f_seq_out:
        f_intermediate = f_seq_out + '.tmp'
    
    # Read patient_code_sequences.txt
    with open(f_pcs_in) as fh:  
        # Skip the heaer line
        fh.readline()
        
        for line in fh:
            # Parse the line into person_id and list of date_occurrences
            pid, date_occurrences = _process_pcs_line(line)
            
            # Keep track of which concepts we can still try to find for this patient
            search_concepts = set(cohort_concepts)

            for date_occurrence in date_occurrences:
                found_concepts = search_concepts.intersection(set(date_occurrence.concept_ids))
                if not found_concepts:           
                    # No matching concepts found. Moving on
                    continue
                    
                # Found at least one desired concept. Create a sequence of occurrences within +/- time_window                                
                date_lower = date_occurrence.date - time_window
                date_upper = date_occurrence.date + time_window
                current_seq = list()
                for do in date_occurrences:
                    if do.date < date_lower:
                        continue
                        
                    if do.date > date_upper:
                        # No more date_occurrences within the desired time window.                         
                        break
                        
                    # The date_occurrence is within the time_window. Add occurrences to seq
                    concepts = do.concept_ids
                    if randomize_order:
                        # Randomize the order of concepts occurring on the same date. Shuffle is applied in place
                        shuffle(concepts)
                    current_seq += concepts
                    
                # Convert the sequence of OMOP concept IDs to TaggedDocument for D2V processing
                tagged_doc_seq = TaggedDocument(words=[str(x) for x in current_seq], tags=[pid])                
                    
                # Save the sequence along with patient ID, time window, and label
                # If more than one matching concept found in this date_occurrence, write them both
                for found_concept in found_concepts:
                    cps = CohortPatientSeq(pid, tagged_doc_seq, found_concept, date_lower, date_upper)
                    cpss.append(cps)
                
                # Remove the found concepts from the list to search
                search_concepts = search_concepts - found_concepts                                              
                if len(search_concepts) == 0:
                    # No more concepts to search for for this patient
                    break

            # Display progress
            count += 1
            if count % 100000 == 0:
                if verbose: 
                    # Processing time
                    ellapsed_time = (time() - t1) / 60
                    print(f'{count} - {ellapsed_time:.01f} min')

                if save_intermediates and f_seq_out:
                    # Save a backup copy of the data
                    pickle.dump(cpss, open(f_intermediate, 'wb'), protocol=pickle.HIGHEST_PROTOCOL)      

    if f_seq_out:
        # Save the concept age distributions            
        pickle.dump(cpss, open(f_seq_out, 'wb'), protocol=pickle.HIGHEST_PROTOCOL)

        # Delete the backup file
        if save_intermediates and path.exists(f_intermediate):
            os.remove(f_intermediate)

    # Display overall processing time
    ellapsed_time = (time() - t1) / 60
    print(f'{count} - {ellapsed_time:.01f} min')
    
    return cpss

### Using chronic kidney disease as an example disease domain
Automatic selection of known disease subtypes of chronic kidney disease (OMOP:46271022) by selecting all observed concept descendants (using OMOP concept_ancestor table) of CKD with min_levels_of_separation > 0 and max_levels_of_separation <= 1  
  
OMOP ID: Condition concept name  
443614: Chronic kidney disease stage 1	
443601: Chronic kidney disease stage 2	
443597: Chronic kidney disease stage 3	
443612: Chronic kidney disease stage 4	
443611: Chronic kidney disease stage 5	
43531578: Chronic kidney disease due to type 2 diabetes mellitus	
44782429: Chronic kidney disease due to hypertension  
  
Note: 44782429 was excluded from data extraction since it's considered an iatrogenic code.

In [ ]:
concepts_ckd = [443614, 443601, 443597, 443612, 443611, 43531578]
f_seq_out = path.join(dir_data, 'ckd_cohort_patient_sequences.pkl')
cpss = create_cohort_patient_sequences(file_pcs, cohort_concepts=concepts_ckd, f_seq_out=f_seq_out, time_window=180, 
                                       randomize_order=True, verbose=True, save_intermediates=False)
n_cpss = len(cpss)
print(n_cpss)

In [ ]:
# See how many examples of each concept we found
cohort_counter = Counter()
for cps in cpss:
    cohort_counter[cps.label] += 1
print(cohort_counter)
print(len(cohort_counter))

In [ ]:
# Count the number of distinct patients
n_patients = len(set([cps.person_id for cps in cpss]))

In [ ]:
# Re-load this data if kernel restarted
with open(path.join(dir_data, 'ckd_cohort_patient_sequences.pkl'), 'rb') as f:
    cpss = pickle.load(f)

# Create patient vectors for cohort 

In [ ]:
# Number of times to run inference on a patient sequence. Mean of inferred vectors will be used
n_inferences = 1

t1 = time()

# Store the patient vectors in a list. The list should have the same order as cpss
patient_vectors = list()
for i, cps in enumerate(cpss):
    l_dm = list()
    l_dbow = list()
    
    # Take the mean of n_inferences iterations of inferring the vector
    for _ in range(n_inferences):
        l_dm.append(model_dm.infer_vector(cps.sequence.words))
        l_dbow.append(model_dm.infer_vector(cps.sequence.words))
    a_dm = np.mean(np.array(l_dm), axis=0)
    a_dbow = np.mean(np.array(l_dm), axis=0)
    patient_vectors.append(np.concatenate((a_dm, a_dbow)))
    
    if i % 10000 == 0:
        ellapsed_time = (time() - t1) / 60
        print(f'{i}: {ellapsed_time:.02f} min')
        
# Convert to numpy array
patient_vectors = np.array(patient_vectors)
        
with open(path.join(dir_data, 'ckd_cohort_patient_vectors.pkl'), 'wb') as f:
    pickle.dump(patient_vectors, f, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
# Re-load this data if kernel restarted
with open(path.join(dir_data, 'ckd_cohort_patient_vectors.pkl'), 'rb') as f:
    patient_vectors = pickle.load(f)

# t-SNE using the defined disease subtypes

In [ ]:
t1 = time()

tsne = TSNE(n_components=2, metric='cosine', perplexity=50, learning_rate=400, init='pca', 
            n_iter=2000, n_jobs=8, verbose=2, random_state=42)
patient_vectors_tsne = tsne.fit_transform(patient_vectors)

ellapsed_time = (time() - t1) / 60
print(f'{ellapsed_time} min')

In [ ]:
# Use the sns bright palette but re-order it so that it goes from purple to red for Stage 1 - 5
palette = sns.color_palette('bright', 6)
palette_ckd = [palette[i] for i in [2, 4, 1, 5, 0, 3]]

# Clustering
Tried DBSCAN and K-means. Could not find good settings for eps and min_samples with DBSCAN that produced good clustering. K-means worked relatively well. 

### DBSCAN

In [ ]:
# Test different settings
for min_samples in [5, 10, 20]:
    for eps in [0.35]:
        dbscan = DBSCAN(eps=eps, min_samples=min_samples, metric='cosine', n_jobs=8)
        clustering = dbscan.fit(patient_vectors)

        counter = Counter()
        for l in clustering.labels_:
            counter[l] += 1

        print(f'eps: {eps}, min_samples: {min_samples}')
        print(counter)

In [ ]:
dbscan = DBSCAN(eps=0.35, min_samples=5, metric='cosine', n_jobs=8)
clustering = dbscan.fit(patient_vectors)

palette_kmeans = sns.color_palette('bright', len(set(clustering.labels_)))
plt.figure(figsize=(16,12))
sns.scatterplot(patient_vectors_tsne[:,0], patient_vectors_tsne[:,1], 
                hue=clustering.labels_, legend='full', palette=palette_kmeans)

### K-means

In [ ]:
kmeans = KMeans(n_clusters=7, verbose=1, random_state=42, n_jobs=8)
clustering = kmeans.fit(patient_vectors)
predicted_labels = clustering.predict(patient_vectors)

In [ ]:
palette_kmeans = sns.color_palette('bright', 7)
plt.figure(figsize=(16,12))
sns.scatterplot(patient_vectors_tsne[:,0], patient_vectors_tsne[:,1], 
                hue=predicted_labels, legend='full', palette=palette_kmeans)

# Disease Profiles

### Known Disease Subtypes

In [ ]:
# Build a list of patient sequences for each known CKD subtype
patients_known_disease_subtype = defaultdict(list)
for cps in cpss:
    patients_known_disease_subtype[cps.label].append(cps)

# dict[disease subtype concept_id] => dict[domain_id] => dict[concept_id] => Counter
known_disease_profiles = dict()

# For each disease subtype, count the number of patients each concept is observed in
for known_disease_subtype, cohort_cpss in patients_known_disease_subtype.items():
    # dict[domain_id] => Counter
    cohort_domain_concept_counter = defaultdict(Counter)
    
    for cohort_cps in cohort_cpss:
        # cohort_cps.sequence is a TaggedDoc with sequence of concepts stored as strings. 
        # Convert it to a list of concept_ids (ints)
        seq = [int(x) for x in cohort_cps.sequence.words]
        
        # Keep track of which conpepts we've already seen for this patient so we don't add again
        concepts_observed = list()  
        
        for concept in seq:
            if concept in concepts_observed:
                # Already seen this concept for this patient
                continue            
                
            domain = df_concepts.loc[concept, 'domain_id']
            cohort_domain_concept_counter[domain][concept] += 1
            concepts_observed.append(concept)
            
    known_disease_profiles[known_disease_subtype] = cohort_domain_concept_counter    

In [ ]:
# Show the stats of each concept that is observed in > 20% of each disease subtype
print('Note: subtype percentages can add up to > 100% since different timelines can be extracted from each patient to represent a disease subtype')
for disease_subtype, cohort_domain_concept_counter in known_disease_profiles.items():        
    concept_name = df_concepts.loc[disease_subtype, 'concept_name']
    n_disease_subtype = len(patients_known_disease_subtype[disease_subtype])
    print(f'{concept_name}: {n_disease_subtype} / {n_patients} = {(n_disease_subtype / n_patients * 100):.02f}')
    
    for domain, concept_counter in cohort_domain_concept_counter.items():
        print(domain)
        
        # Create a DataFrame from the counts
        df_subtype_domain_counts = pd.DataFrame.from_dict(concept_counter, orient='index', columns=['count'])
        df_subtype_domain_counts.index.name = 'concept_id'
        
        # Calculate prevalence of each concept within the disease subtype
        df_subtype_domain_counts['prevalence'] = df_subtype_domain_counts['count'] / n_disease_subtype * 100
        
        # Add concept_name to DataFrame
        df_subtype_domain_counts = df_subtype_domain_counts.join(df_concepts['concept_name'], how='left')
        
        # Re-arrange column order,  sort by descending order of count, and display
        df_subtype_domain_counts = df_subtype_domain_counts[['concept_name', 'count', 'prevalence']]
        df_subtype_domain_counts.sort_values(by='count', ascending=False, inplace=True)
        display(df_subtype_domain_counts.loc[((df_subtype_domain_counts['prevalence'] > 20) & (df_subtype_domain_counts['count'] > 20)), :].head(20))
        

### Learned Disease Subtypes

In [ ]:
# Build a list of patient sequences for each predicted (learned) disease subtype
patients_learned_disease_subtype = defaultdict(list)
for i, cps in enumerate(cpss):
    patients_learned_disease_subtype[predicted_labels[i]].append(cps)

# dict[disease subtype concept_id] => dict[domain_id] => dict[concept_id] => Counter
learned_disease_profiles = dict()

# For each disease subtype, count the number of patients each concept is observed in
for disease_subtype, cohort_cpss in patients_learned_disease_subtype.items():
    # dict[domain_id] => Counter
    cohort_domain_concept_counter = defaultdict(Counter)
    
    for cohort_cps in cohort_cpss:
        # cohort_cps.sequence is a TaggedDoc with sequence of concepts stored as strings. 
        # Convert it to a list of concept_ids (ints)
        seq = [int(x) for x in cohort_cps.sequence.words]
        
        # Keep track of which conpepts we've already seen for this patient so we don't add again
        concepts_observed = list()  
        
        for concept in seq:
            if concept in concepts_observed:
                # Already seen this concept for this patient
                continue            
                
            domain = df_concepts.loc[concept, 'domain_id']
            cohort_domain_concept_counter[domain][concept] += 1
            concepts_observed.append(concept)
            
    learned_disease_profiles[disease_subtype] = cohort_domain_concept_counter    

In [ ]:
# Show the stats of each concept that is observed in > 20% of each disease subtype
print('Note: subtype percentages can add up to > 100% since different timelines can be extracted from each patient to represent a disease subtype')
for disease_subtype, cohort_domain_concept_counter in learned_disease_profiles.items():        
    n_disease_subtype = len(patients_learned_disease_subtype[disease_subtype])
    print(f'{disease_subtype}: {n_disease_subtype} / {n_patients} = {(n_disease_subtype / n_patients * 100):.02f}')
    
    for domain, concept_counter in cohort_domain_concept_counter.items():
        print(domain)
        
        # Create a DataFrame from the counts
        df_subtype_domain_counts = pd.DataFrame.from_dict(concept_counter, orient='index', columns=['count'])
        df_subtype_domain_counts.index.name = 'concept_id'
        
        # Calculate prevalence of each concept within the disease subtype
        df_subtype_domain_counts['prevalence'] = df_subtype_domain_counts['count'] / n_disease_subtype * 100
        
        # Add concept_name to DataFrame
        df_subtype_domain_counts = df_subtype_domain_counts.join(df_concepts['concept_name'], how='left')
        
        # Re-arrange column order,  sort by descending order of count, and display
        df_subtype_domain_counts = df_subtype_domain_counts[['concept_name', 'count', 'prevalence']]
        df_subtype_domain_counts.sort_values(by='count', ascending=False, inplace=True)
        display(df_subtype_domain_counts.loc[((df_subtype_domain_counts['prevalence'] > 20) & (df_subtype_domain_counts['count'] > 20)), :].head(20))
        